In [1]:
from itertools import product
import random
from gurobipy import Model, GRB, quicksum
import numpy as np
import pandas as pd

random.seed(42)

def complement_pattern(pattern):
    """Return the complement of a pattern by swapping 'H' and 'A'."""
    return ''.join('H' if c == 'A' else 'A' for c in pattern)

def valid_pattern(pattern):
    # Count occurrences of 'H' and 'A'
    h_count = pattern.count('H')
    a_count = pattern.count('A')
    n = len(pattern)
    
    # For odd length n, majority should be (n+1)/2 and minority (n-1)/2
    majority_count = (n + 1) // 2
    minority_count = (n - 1) // 2

    # Check the balancing condition: either H is majority and A is minority or vice versa.
    if not ((h_count == majority_count and a_count == minority_count) or 
            (h_count == minority_count and a_count == majority_count)):
        return False

    # Check for more than two consecutive identical symbols (no three in a row)
    for i in range(n - 2):
        if pattern[i] == pattern[i+1] == pattern[i+2]:
            return False

    return True

def generate_subset_with_complements(n=11, keep_probability=1.0):
    """
    Generate a subset of valid patterns of length n, ensuring that
    if a pattern is chosen, its complement is also chosen.
    
    :param n: length of the pattern (default 11).
    :param keep_probability: fraction of pairs to keep (0.0 to 1.0).
    :return: list of chosen patterns (each accompanied by its complement).
    """
    chosen_patterns = []
    seen = set()  # to avoid re-checking pairs

    for p_tuple in product('HA', repeat=n):
        p = ''.join(p_tuple)

        # Already handled if we or our complement is in 'seen'
        if p in seen:
            continue

        c = complement_pattern(p)

        # Enforce p < c lexicographically so we only handle each pair once
        if p < c:
            # Check validity of p (its complement is automatically valid
            # under these constraints)
            if valid_pattern(p):
                # Randomly decide whether to keep this pair
                if random.random() < keep_probability:
                    chosen_patterns.append(p)
                    chosen_patterns.append(c)
            
            # Mark both as seen
            seen.add(p)
            seen.add(c)
        else:
            # The pair will be handled when we reach c (if c < p)
            # so do nothing here
            pass

    return chosen_patterns


# Example usage:
if __name__ == "__main__":
    # Generate a small random subset of valid patterns (say 10%).
    # Each chosen pattern is paired with its complement.
    subset = generate_subset_with_complements(n=19, keep_probability=0.02)

    print(f"Number of chosen patterns: {len(subset)}")
    # Print the first few pairs (p, c)
    for i in range(0, min(len(subset), 10), 2):
        print(subset[i], "<->", subset[i+1])


Number of chosen patterns: 164
AHHAHHAHHAAHHAAHHAA <-> HAAHAAHAAHHAAHHAAHH
AHHAHHAHAAHHAAHAHAH <-> HAAHAAHAHHAAHHAHAHA
AHHAHHAHAAHAAHHAHAA <-> HAAHAAHAHHAHHAAHAHH
AHHAHHAHAAHAAHHAAHA <-> HAAHAAHAHHAHHAAHHAH
AHHAHAHHAHAHAHAHAHA <-> HAAHAHAAHAHAHAHAHAH


In [2]:
valid_patterns = subset

In [3]:
# Function to check if two patterns are complementary
def are_complementary(pattern1, pattern2):
    return all((c1 == 'H' and c2 == 'A') or (c1 == 'A' and c2 == 'H') for c1, c2 in zip(pattern1, pattern2))

# Find all pairs of complementary patterns
C = []
for i in range(len(valid_patterns)):
    for j in range(i + 1, len(valid_patterns)):
        if are_complementary(valid_patterns[i], valid_patterns[j]):
            C.append((valid_patterns[i], valid_patterns[j]))

print(C)

C_indices = [(valid_patterns.index(x), valid_patterns.index(y)) for x, y in C]

print(C_indices)


[('AHHAHHAHHAAHHAAHHAA', 'HAAHAAHAAHHAAHHAAHH'), ('AHHAHHAHAAHHAAHAHAH', 'HAAHAAHAHHAAHHAHAHA'), ('AHHAHHAHAAHAAHHAHAA', 'HAAHAAHAHHAHHAAHAHH'), ('AHHAHHAHAAHAAHHAAHA', 'HAAHAAHAHHAHHAAHHAH'), ('AHHAHAHHAHAHAHAHAHA', 'HAAHAHAAHAHAHAHAHAH'), ('AHHAHAHHAHAAHAHAHAH', 'HAAHAHAAHAHHAHAHAHA'), ('AHHAHAHHAHAAHAAHAHA', 'HAAHAHAAHAHHAHHAHAH'), ('AHHAHAHAHHAAHHAAHAH', 'HAAHAHAHAAHHAAHHAHA'), ('AHHAHAHAHAHHAAHAHAA', 'HAAHAHAHAHAAHHAHAHH'), ('AHHAHAHAHAAHHAHHAHA', 'HAAHAHAHAHHAAHAAHAH'), ('AHHAHAHAAHHAAHAHHAA', 'HAAHAHAHHAAHHAHAAHH'), ('AHHAHAHAAHAAHHAHHAA', 'HAAHAHAHHAHHAAHAAHH'), ('AHHAHAAHHAAHAHAHAAH', 'HAAHAHHAAHHAHAHAHHA'), ('AHHAHAAHAHAHHAHHAHA', 'HAAHAHHAHAHAAHAAHAH'), ('AHHAHAAHAAHHAHHAHAA', 'HAAHAHHAHHAAHAAHAHH'), ('AHHAAHHAAHAHAHAHAHH', 'HAAHHAAHHAHAHAHAHAA'), ('AHHAAHAHHAAHHAHAHAH', 'HAAHHAHAAHHAAHAHAHA'), ('AHHAAHAHAHAHAHHAHAH', 'HAAHHAHAHAHAHAAHAHA'), ('AHHAAHAHAHAHAHAHAAH', 'HAAHHAHAHAHAHAHAHHA'), ('AHHAAHAAHHAAHAHHAHH', 'HAAHHAHHAAHHAHAAHAA'), ('AHHAAHAAHAHAHAHHAAH', 'HAAHHAHHAHAHAH

In [4]:
solution_found = {'r': {(0, 1, 0): 1.0,
  (0, 2, 8): 1.0,
  (0, 3, 1): 1.0,
  (0, 4, 10): 1.0,
  (0, 5, 2): 1.0,
  (0, 6, 11): 1.0,
  (0, 7, 3): 1.0,
  (0, 8, 17): 1.0,
  (0, 9, 15): 1.0,
  (0, 10, 9): 1.0,
  (0, 11, 4): 1.0,
  (0, 12, 16): 1.0,
  (0, 13, 18): 1.0,
  (0, 14, 5): 1.0,
  (0, 15, 14): 1.0,
  (0, 16, 7): 1.0,
  (0, 17, 13): 1.0,
  (0, 18, 6): 1.0,
  (0, 19, 12): 1.0,
  (1, 2, 17): 1.0,
  (1, 3, 10): 1.0,
  (1, 4, 18): 1.0,
  (1, 5, 14): 1.0,
  (1, 6, 9): 1.0,
  (1, 7, 12): 1.0,
  (1, 8, 4): 1.0,
  (1, 9, 8): 1.0,
  (1, 10, 16): 1.0,
  (1, 11, 11): 1.0,
  (1, 12, 13): 1.0,
  (1, 13, 5): 1.0,
  (1, 14, 3): 1.0,
  (1, 15, 15): 1.0,
  (1, 16, 1): 1.0,
  (1, 17, 7): 1.0,
  (1, 18, 2): 1.0,
  (1, 19, 6): 1.0,
  (2, 3, 7): 1.0,
  (2, 4, 11): 1.0,
  (2, 5, 10): 1.0,
  (2, 6, 18): 1.0,
  (2, 7, 15): 1.0,
  (2, 8, 14): 1.0,
  (2, 9, 1): 1.0,
  (2, 10, 12): 1.0,
  (2, 11, 2): 1.0,
  (2, 12, 6): 1.0,
  (2, 13, 13): 1.0,
  (2, 14, 9): 1.0,
  (2, 15, 0): 1.0,
  (2, 16, 5): 1.0,
  (2, 17, 4): 1.0,
  (2, 18, 16): 1.0,
  (2, 19, 3): 1.0,
  (3, 4, 3): 1.0,
  (3, 5, 13): 1.0,
  (3, 6, 2): 1.0,
  (3, 7, 17): 1.0,
  (3, 8, 12): 1.0,
  (3, 9, 5): 1.0,
  (3, 10, 15): 1.0,
  (3, 11, 6): 1.0,
  (3, 12, 4): 1.0,
  (3, 13, 14): 1.0,
  (3, 14, 16): 1.0,
  (3, 15, 11): 1.0,
  (3, 16, 9): 1.0,
  (3, 17, 18): 1.0,
  (3, 18, 0): 1.0,
  (3, 19, 8): 1.0,
  (4, 5, 8): 1.0,
  (4, 6, 17): 1.0,
  (4, 7, 4): 1.0,
  (4, 8, 9): 1.0,
  (4, 9, 2): 1.0,
  (4, 10, 5): 1.0,
  (4, 11, 16): 1.0,
  (4, 12, 15): 1.0,
  (4, 13, 7): 1.0,
  (4, 14, 14): 1.0,
  (4, 15, 1): 1.0,
  (4, 16, 6): 1.0,
  (4, 17, 12): 1.0,
  (4, 18, 13): 1.0,
  (4, 19, 0): 1.0,
  (5, 6, 3): 1.0,
  (5, 7, 16): 1.0,
  (5, 8, 1): 1.0,
  (5, 9, 6): 1.0,
  (5, 10, 11): 1.0,
  (5, 11, 9): 1.0,
  (5, 12, 18): 1.0,
  (5, 13, 17): 1.0,
  (5, 14, 0): 1.0,
  (5, 15, 7): 1.0,
  (5, 16, 12): 1.0,
  (5, 17, 5): 1.0,
  (5, 18, 4): 1.0,
  (5, 19, 15): 1.0,
  (6, 7, 0): 1.0,
  (6, 8, 10): 1.0,
  (6, 9, 16): 1.0,
  (6, 10, 6): 1.0,
  (6, 11, 1): 1.0,
  (6, 12, 5): 1.0,
  (6, 13, 8): 1.0,
  (6, 14, 7): 1.0,
  (6, 15, 4): 1.0,
  (6, 16, 13): 1.0,
  (6, 17, 15): 1.0,
  (6, 18, 12): 1.0,
  (6, 19, 14): 1.0,
  (7, 8, 8): 1.0,
  (7, 9, 9): 1.0,
  (7, 10, 7): 1.0,
  (7, 11, 10): 1.0,
  (7, 12, 2): 1.0,
  (7, 13, 6): 1.0,
  (7, 14, 1): 1.0,
  (7, 15, 5): 1.0,
  (7, 16, 18): 1.0,
  (7, 17, 11): 1.0,
  (7, 18, 14): 1.0,
  (7, 19, 13): 1.0,
  (8, 9, 13): 1.0,
  (8, 10, 18): 1.0,
  (8, 11, 3): 1.0,
  (8, 12, 11): 1.0,
  (8, 13, 0): 1.0,
  (8, 14, 15): 1.0,
  (8, 15, 6): 1.0,
  (8, 16, 16): 1.0,
  (8, 17, 2): 1.0,
  (8, 18, 7): 1.0,
  (8, 19, 5): 1.0,
  (9, 10, 4): 1.0,
  (9, 11, 12): 1.0,
  (9, 12, 0): 1.0,
  (9, 13, 11): 1.0,
  (9, 14, 10): 1.0,
  (9, 15, 18): 1.0,
  (9, 16, 3): 1.0,
  (9, 17, 14): 1.0,
  (9, 18, 17): 1.0,
  (9, 19, 7): 1.0,
  (10, 11, 14): 1.0,
  (10, 12, 17): 1.0,
  (10, 13, 3): 1.0,
  (10, 14, 13): 1.0,
  (10, 15, 2): 1.0,
  (10, 16, 10): 1.0,
  (10, 17, 0): 1.0,
  (10, 18, 8): 1.0,
  (10, 19, 1): 1.0,
  (11, 12, 7): 1.0,
  (11, 13, 15): 1.0,
  (11, 14, 17): 1.0,
  (11, 15, 13): 1.0,
  (11, 16, 0): 1.0,
  (11, 17, 8): 1.0,
  (11, 18, 5): 1.0,
  (11, 19, 18): 1.0,
  (12, 13, 12): 1.0,
  (12, 14, 8): 1.0,
  (12, 15, 3): 1.0,
  (12, 16, 14): 1.0,
  (12, 17, 1): 1.0,
  (12, 18, 9): 1.0,
  (12, 19, 10): 1.0,
  (13, 14, 2): 1.0,
  (13, 15, 16): 1.0,
  (13, 16, 4): 1.0,
  (13, 17, 10): 1.0,
  (13, 18, 1): 1.0,
  (13, 19, 9): 1.0,
  (14, 15, 12): 1.0,
  (14, 16, 11): 1.0,
  (14, 17, 6): 1.0,
  (14, 18, 18): 1.0,
  (14, 19, 4): 1.0,
  (15, 16, 8): 1.0,
  (15, 17, 9): 1.0,
  (15, 18, 10): 1.0,
  (15, 19, 17): 1.0,
  (16, 17, 17): 1.0,
  (16, 18, 15): 1.0,
  (16, 19, 2): 1.0,
  (17, 18, 3): 1.0,
  (17, 19, 16): 1.0,
  (18, 19, 11): 1.0},
 'y': {0: 1.0,
  1: 1.0,
  2: 1.0,
  3: 1.0,
  4: 1.0,
  5: 1.0,
  6: 1.0,
  7: 1.0,
  8: 1.0,
  9: 1.0,
  10: 1.0,
  11: 1.0,
  12: 1.0,
  13: 1.0,
  14: 1.0,
  15: 1.0,
  16: 1.0,
  17: 1.0,
  18: 1.0,
  19: 1.0}}

In [5]:
P_hat = solution_found['y']

In [6]:
# Dictionary `feasible_opponents` where feasible_opponents[i, k]
# is the set of feasible opponents for pattern i in week t


# Initialize the feasible opponents dictionary
feasible_opponents = {}

# Iterate through each pattern and week
num_weeks = len(valid_patterns[0])

for i in P_hat:
    for k in range(num_weeks):
        # Get the H/A status of pattern i in week k
        current_status = valid_patterns[i][k]
        
        # Find all patterns that have the opposite status in week k
        feasible_opponents[i, k] = {j for j in P_hat if j != i and valid_patterns[j][k] != current_status}


# Print the feasible opponents dictionary for verification
for key, value in feasible_opponents.items():
    print(f"Pattern {key[0]} in week {key[1]} can play against patterns {value}")


Pattern 0 in week 0 can play against patterns {1, 3, 5, 7, 9, 11, 13, 15, 17, 19}
Pattern 0 in week 1 can play against patterns {1, 3, 5, 7, 9, 11, 13, 15, 17, 19}
Pattern 0 in week 2 can play against patterns {1, 3, 5, 7, 9, 11, 13, 15, 17, 19}
Pattern 0 in week 3 can play against patterns {1, 3, 5, 7, 9, 11, 13, 15, 17, 19}
Pattern 0 in week 4 can play against patterns {1, 3, 5, 7, 9, 11, 13, 15, 17, 19}
Pattern 0 in week 5 can play against patterns {1, 3, 5, 7, 8, 10, 12, 14, 16, 18}
Pattern 0 in week 6 can play against patterns {1, 3, 5, 7, 8, 10, 12, 14, 16, 18}
Pattern 0 in week 7 can play against patterns {1, 3, 5, 7, 9, 11, 13, 14, 16, 18}
Pattern 0 in week 8 can play against patterns {1, 2, 4, 6, 8, 10, 12, 15, 17, 19}
Pattern 0 in week 9 can play against patterns {1, 3, 5, 7, 8, 10, 12, 14, 17, 19}
Pattern 0 in week 10 can play against patterns {1, 2, 4, 6, 9, 11, 13, 15, 16, 19}
Pattern 0 in week 11 can play against patterns {1, 3, 4, 6, 9, 10, 12, 14, 17, 19}
Pattern 0 in w

In [ ]:
# Create the list of teams and ranking
teams = [
    "Manchester City", "Liverpool", "Arsenal", 
    "Manchester United", "Chelsea", "Tottenham Hotspur", 
    "Newcastle United", "Brighton", "Aston Villa", "West Ham",
    "Real Madrid", "Barcelona",
    "Juventus", "Inter Milan", "AC Milan", "Paris Saint-Germain", 
    "Bayern Munich", "Borussia Dortmund", "Ajax", "Atletico Madrid"
]


ranking = {
    1: teams[:6],    # Position 1
    2: teams[6:12],  # Position 2
    3: teams[12:]    # Position 3
}

# Assign a unique number to each team
team_numbers = {i : team for i, team in enumerate(teams)}

# Create a reverse mapping to look up the number of a team
team_to_number = {team: number for number, team in team_numbers.items()}

# Create a dictionary to store subsets of stronger teams
stronger_teams = {}

# Iterate through each team and determine stronger teams
for position, teams_at_position in ranking.items():
    for team in teams_at_position:
        team_number = team_to_number[team]  # Get the team's unique number
        stronger_teams[team_number] = [
            team_to_number[stronger_team]
            for higher_position in range(1, position)  # Look at higher positions only
            for stronger_team in ranking[higher_position]
        ]

# Set of teams located in the same town
D = [(0,5)]

# Print the team-to-number mapping and the stronger teams dictionary
print("Team Numbers:", team_numbers)
print("Stronger Teams:", stronger_teams)
print('Pairs of teams located in the same town', D)

Team Numbers: {0: 'Manchester City', 1: 'Liverpool', 2: 'Arsenal', 3: 'Manchester United', 4: 'Chelsea', 5: 'Tottenham Hotspur', 6: 'Newcastle United', 7: 'Brighton', 8: 'Aston Villa', 9: 'West Ham', 10: 'Real Madrid', 11: 'Barcelona', 12: 'Juventus', 13: 'Inter Milan', 14: 'AC Milan', 15: 'Paris Saint-Germain', 16: 'Bayern Munich', 17: 'Borussia Dortmund', 18: 'Ajax', 19: 'Atletico Madrid'}
Stronger Teams: {0: [], 1: [], 2: [], 3: [], 4: [], 5: [], 6: [0, 1, 2, 3, 4, 5], 7: [0, 1, 2, 3, 4, 5], 8: [0, 1, 2, 3, 4, 5], 9: [0, 1, 2, 3, 4, 5], 10: [0, 1, 2, 3, 4, 5], 11: [0, 1, 2, 3, 4, 5], 12: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11], 13: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11], 14: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11], 15: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11], 16: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11], 17: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11], 18: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11], 19: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]}
Pairs of teams located in the same town [(0, 5)]


In [8]:
from gurobipy import Model, GRB, quicksum

# Initialize the model
m = Model("Team_to_Pattern_Allocation")

# Parameters
T = range(20)       # Teams (i, j)
W = range(19)       # Weeks (w)
W_minus_1 = range(18)  # Weeks for carry-over pairs (w, w+1)


## Decision Variables

# r_var[p, pp, w] = 1 if pattern p is paired with pattern pp in week w
r_var = m.addVars(P_hat, P_hat, W, vtype=GRB.BINARY, name="r")

# y_var[p] = 1 if pattern p is selected among the set
y_var = m.addVars(P_hat, vtype=GRB.BINARY, name="y")

# z_var[i, p] = 1 if team i is assigned to pattern p
z_var = m.addVars(T, P_hat, vtype=GRB.BINARY, name="z")

# s_var[i, w] = 1 if team i faces stronger opponents in weeks w and w+1
s_var = m.addVars(T, W_minus_1, vtype=GRB.BINARY, name="s")

# x_var[i, j, w] = 1 if team i plays team j in week w
x_var = m.addVars(T, T, W, vtype=GRB.BINARY, name="x")

Set parameter Username
Set parameter LicenseID to value 2622628
Academic license - for non-commercial use only - expires 2026-02-16


In [ ]:

# Select exactly 10 patterns (or |P_hat| patterns)
m.addConstr(quicksum(y_var[p] for p in P_hat) == len(P_hat), name="select_patterns")


# Ensure each pattern used in a week is one of the selected patterns
# For each pattern p and week w, the sum over valid opponents equals y_var[p]

for p in P_hat:
    for w in W:
        if (p, w) in feasible_opponents:
            m.addConstr(
                quicksum(r_var[p, pp, w] for pp in feasible_opponents[(p, w)] if p < pp) +
                quicksum(r_var[pp, p, w] for pp in feasible_opponents[(p, w)] if pp < p)
                == y_var[p],
                name=f"pattern_consistency_{p}_{w}"
            )


# Each pair of selected patterns is selected for at most one week

for p in P_hat:
    for pp in P_hat:
        if p != pp:
            m.addConstr(
                quicksum(r_var[p, pp, w] for w in W) <= y_var[p],
                name=f"unique_match_{p}_{pp}"
            )


# Setting irrelevant r_var variables to 0:
# 1) If p >= pp
# 2) If pp is not a feasible opponent for p in week w

for p in P_hat:
    for pp in P_hat:
        if p >= pp:
            for w in W:
                r_var[p, pp, w].ub = 0

for p in P_hat:
    for w in W:
        for pp in P_hat:
            if p < pp:
                if pp not in feasible_opponents.get((p, w), []):
                    r_var[p, pp, w].ub = 0


# Each team must be assigned to exactly one pattern
for i in T:
    m.addConstr(
        quicksum(z_var[i, phi] for phi in P_hat) == 1,
        name=f"team_pattern_allocation_{i}"
    )


# Each selected pattern must be assigned to exactly one team
for phi in P_hat:
    m.addConstr(
        quicksum(z_var[i, phi] for i in T) == y_var[phi],
        name=f"team_pattern_unique_allocation_{phi}"
    )


# Linking match assignments to team-to-pattern allocations and pattern pairings
# For each week, each team pair (i,j) with i < j, and each pattern pair (phi, phi_prime) with phi < phi_prime,
# enforce: x_var[i,j,w] >= z_var[i,phi] + z_var[i,phi_prime] + z_var[j,phi] + z_var[j,phi_prime] + r_var[phi, phi_prime, w] + r_var[phi_prime, phi, w] - 2

for w in W:
    for i in T:
        for j in T:
            if i < j:
                for phi in P_hat:
                    for phi_prime in P_hat:
                        if phi < phi_prime:
                            m.addConstr(
                                x_var[i, j, w] >=  z_var[i, phi] + z_var[i, phi_prime] +
                                                   z_var[j, phi] + z_var[j, phi_prime] +
                                                   r_var[phi, phi_prime, w] + r_var[phi_prime, phi, w] - 2,
                                name=f"match_link_{i}_{j}_{w}_{phi}_{phi_prime}"
                            )


# Each pair of teams plays exactly one match throughout the season
for i in T:
    for j in T:
        if i < j:
            m.addConstr(
                quicksum(x_var[i, j, w] for w in W) == 1,
                name=f"one_match_{i}_{j}"
            )


# Each team plays exactly one match in each week
for w in W:
    for i in T:
        m.addConstr(
            quicksum(x_var[i, j, w] for j in T if i < j) +
            quicksum(x_var[j, i, w] for j in T if j < i)
            == 1,
            name=f"one_match_per_week_{i}_{w}"
        )


# Setting irrelevant x_var variables to 0 if i >= j

for i in T:
    for j in T:
        if i >= j:
            for w in W:
                x_var[i, j, w].ub = 0


# Constraint 9: Assignment of complementary patterns (same-venue constraint)
# For each team pair (i, j) in D and each complementary pair (c, c_prime) in C_indices,
# enforce: (z_var[i, c] + z_var[i, c_prime]) - (z_var[j, c_prime] + z_var[j, c]) = 0

for (i, j) in D:
    for (c, c_prime) in C_indices:
        if c in P_hat:
            m.addConstr(
                (z_var[i, c] + z_var[i, c_prime]) - (z_var[j, c_prime] + z_var[j, c]) == 0,
                name=f"comp_pattern_{i}_{j}_{c}_{c_prime}"
            )


# Carry-over effect between consecutive weeks
# For each team i and each week w in W_minus_1,
# enforce the carry-over effect constraint based on matches against stronger teams.

for i in T:
    for w in W_minus_1:
        m.addConstr(
            quicksum(x_var[i, j, w] for j in stronger_teams[i] if i < j) +
            quicksum(x_var[j, i, w] for j in stronger_teams[i] if j < i) +
            quicksum(x_var[i, j, w+1] for j in stronger_teams[i] if i < j) +
            quicksum(x_var[j, i, w+1] for j in stronger_teams[i] if j < i)
            <= 1 + s_var[i, w],
            name=f"carry_over_effect_{i}_{w}"
        )
        m.addConstr(
            quicksum(x_var[i, j, w] for j in stronger_teams[i] if i < j) +
            quicksum(x_var[j, i, w] for j in stronger_teams[i] if j < i)
            >= s_var[i, w],
            name=f"carry_over_lower1_{i}_{w}"
        )
        m.addConstr(
            quicksum(x_var[i, j, w+1] for j in stronger_teams[i] if i < j) +
            quicksum(x_var[j, i, w+1] for j in stronger_teams[i] if j < i)
            >= s_var[i, w],
            name=f"carry_over_lower2_{i}_{w}"
        )


# Objective: Minimize total carry-over effect

m.setObjective(quicksum(s_var[i, w] for i in T for w in W_minus_1), GRB.MINIMIZE)



In [10]:

# Solve the model
m.update()

print("Number of constraints:", m.NumConstrs)
print("Number of variables:", m.NumVars)
m.optimize()  


if m.status == GRB.OPTIMAL:
    print("Optimal solution found")
    selected_patterns = {p:     y_var[       p].x for p in P_hat                      if y_var[     p].x > 0.5}
    z_sol = {(i, phi):          z_var[i,   phi].x for i in T     for phi in P_hat     if z_var[i, phi].x > 0.5}
    s_sol = {(i, w):            s_var[i,     w].x for i in T     for w   in W_minus_1 if s_var[i,   w].x > 0.5}
    x_sol = {(i, j, w):         x_var[i,  j, w].x for i in T     for j   in T         for w in W if x_var[i,  j, w].x > 0.5}
    patterns_opt = {(p, pp, w): r_var[p, pp, w].x for p in P_hat for pp  in P_hat     for w in W if r_var[p, pp, w].x > 0.5}
    print("Team-to-pattern allocation:", z_sol)
    print("Carry-over effect:", s_sol)
    print("Matches between teams:", x_sol)
    print("Pattern pairings:", patterns_opt)
    print("Selected patterns:", selected_patterns)
else:
    print("No optimal solution found")


Number of constraints: 688361
Number of variables: 15980
Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (win64 - Windows 11.0 (26100.2))

CPU model: AMD Ryzen 5 5600X 6-Core Processor, instruction set [SSE2|AVX|AVX2]
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 688361 rows, 15980 columns and 4835374 nonzeros
Model fingerprint: 0xc3821ee9
Variable types: 0 continuous, 15980 integer (15980 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+01]
Presolve removed 325415 rows and 9818 columns (presolve time = 5s)...
Presolve removed 325415 rows and 9818 columns
Presolve time: 5.99s
Presolved: 362946 rows, 6162 columns, 2193630 nonzeros
Variable types: 0 continuous, 6162 integer (6162 binary)

Root simplex log...

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0      handle free variables             

Check Patterns for 'H' and 'A' Specifications

In [11]:
pattern_to_team = {team_numbers[team] : p for (team, p), value in z_sol.items() if value > 0.5}
weeks = {}

for (team1, team2, week), value in x_sol.items():
    if value == 1.0: 
        if week not in weeks:
            weeks[week] = []
        key = pattern_to_team[team_numbers[team1]]  
        if valid_patterns[key][week] == 'H':
            match = f"{team_numbers[team1]} vs {team_numbers[team2]}"
        else:
            match = f"{team_numbers[team2]} vs {team_numbers[team1]}"
        if match not in weeks[week]:
            weeks[week].append(match)




Display Final Output

In [12]:

# Sort the weeks to ensure correct order
sorted_weeks_example = sorted(weeks.keys())

# Find the maximum number of matches in a week to standardize row count
max_matches_example = max(len(matches) for matches in weeks.values())

# Create a DataFrame with rows as matches and columns as weeks
formatted_data_example = {
    f"Week {week}": weeks[week] + [None] * (max_matches_example - len(weeks[week]))
    for week in sorted_weeks_example
}

formatted_df_example = pd.DataFrame(formatted_data_example)
formatted_df_example

,Week 0,Week 1,Week 2,Week 3,Week 4,Week 5,Week 6,Week 7,Week 8,Week 9,Week 10,Week 11,Week 12,Week 13,Week 14,Week 15,Week 16,Week 17,Week 18
0,Manchester City vs Borussia Dortmund,Tottenham Hotspur vs Manchester City,Manchester United vs Manchester City,Manchester City vs Ajax,Chelsea vs Manchester City,Manchester City vs Arsenal,Liverpool vs Manchester City,Paris Saint-Germain vs Manchester City,Manchester City vs Barcelona,Aston Villa vs Manchester City,Manchester City vs Brighton,Newcastle United vs Manchester City,Manchester City vs Juventus,AC Milan vs Manchester City,Manchester City vs Atletico Madrid,West Ham vs Manchester City,Manchester City vs Bayern Munich,Real Madrid vs Manchester City,Manchester City vs Inter Milan
1,Liverpool vs Chelsea,Newcastle United vs Liverpool,Tottenham Hotspur vs Liverpool,Liverpool vs Real Madrid,Ajax vs Liverpool,Inter Milan vs Liverpool,Arsenal vs Manchester United,Borussia Dortmund vs Liverpool,Brighton vs Liverpool,Liverpool vs Juventus,Liverpool vs West Ham,Paris Saint-Germain vs Liverpool,Atletico Madrid vs Liverpool,Liverpool vs Manchester United,Liverpool vs Aston Villa,Arsenal vs Liverpool,AC Milan vs Liverpool,Liverpool vs Barcelona,Liverpool vs Bayern Munich
2,Inter Milan vs Arsenal,Arsenal vs Barcelona,Arsenal vs Paris Saint-Germain,Aston Villa vs Arsenal,Arsenal vs AC Milan,Manchester United vs Newcastle United,Chelsea vs Inter Milan,Juventus vs Arsenal,Arsenal vs Atletico Madrid,Bayern Munich vs Arsenal,Ajax vs Arsenal,Arsenal vs Borussia Dortmund,Arsenal vs Real Madrid,Tottenham Hotspur vs Arsenal,Arsenal vs Brighton,Manchester United vs Borussia Dortmund,Newcastle United vs Arsenal,Arsenal vs Chelsea,West Ham vs Arsenal
3,AC Milan vs Manchester United,Manchester United vs Brighton,Chelsea vs Bayern Munich,Barcelona vs Manchester United,Manchester United vs Bayern Munich,Barcelona vs Chelsea,Tottenham Hotspur vs Paris Saint-Germain,Manchester United vs West Ham,Manchester United vs Ajax,Atletico Madrid vs Manchester United,Real Madrid vs Manchester United,Manchester United vs Chelsea,Manchester United vs Inter Milan,Chelsea vs Borussia Dortmund,Juventus vs Manchester United,Atletico Madrid vs Chelsea,Manchester United vs Paris Saint-Germain,Tottenham Hotspur vs Manchester United,Aston Villa vs Manchester United
4,Aston Villa vs Tottenham Hotspur,Chelsea vs West Ham,Newcastle United vs Inter Milan,Brighton vs Chelsea,Tottenham Hotspur vs Inter Milan,Juventus vs Tottenham Hotspur,Newcastle United vs Ajax,Tottenham Hotspur vs Chelsea,Chelsea vs Real Madrid,Chelsea vs Newcastle United,Paris Saint-Germain vs Chelsea,Tottenham Hotspur vs Atletico Madrid,Chelsea vs Ajax,Barcelona vs Newcastle United,AC Milan vs Chelsea,Tottenham Hotspur vs Real Madrid,Chelsea vs Aston Villa,Atletico Madrid vs Newcastle United,Chelsea vs Juventus
5,Paris Saint-Germain vs Newcastle United,Real Madrid vs Aston Villa,Borussia Dortmund vs Brighton,West Ham vs Tottenham Hotspur,Newcastle United vs Aston Villa,AC Milan vs Brighton,Brighton vs Barcelona,Real Madrid vs Newcastle United,Bayern Munich vs Tottenham Hotspur,Tottenham Hotspur vs AC Milan,Barcelona vs Tottenham Hotspur,Brighton vs Bayern Munich,Brighton vs Tottenham Hotspur,Real Madrid vs Brighton,Newcastle United vs Tottenham Hotspur,Brighton vs Newcastle United,Ajax vs Tottenham Hotspur,West Ham vs Brighton,Borussia Dortmund vs Tottenham Hotspur
6,Brighton vs Ajax,Juventus vs AC Milan,Atletico Madrid vs Aston Villa,Bayern Munich vs Newcastle United,Juventus vs Brighton,Aston Villa vs West Ham,Bayern Munich vs Aston Villa,Aston Villa vs Brighton,Newcastle United vs Juventus,Brighton vs Paris Saint-Germain,Newcastle United vs Borussia Dortmund,Inter Milan vs Aston Villa,West Ham vs Newcastle United,Aston Villa vs Paris Saint-Germain,Paris Saint-Germain vs West Ham,Aston Villa vs Ajax,Brighton vs Inter Milan,Aston Villa vs Borussia Dortmund,AC Milan vs Newcastle United
7,West Ham vs Juventus,Borussia Dortmund vs Inter Milan,Ajax vs West Ham,Par